## Patch creation function(old)

Creating 4 patches of 512 from 1024 patch. H5->png

| Sub-Patch    | Offset (x, y) | Absolute Coord | Filename Output                   |
| ------------ | ------------- | -------------- | --------------------------------- |
| Top-Left     | (0, 0)        | (2016, 19456)  | `..._x2016_y19456_patchXXXXX.png` |
| Top-Right    | (512, 0)      | (2528, 19456)  | `..._x2528_y19456_patchXXXXX.png` |
| Bottom-Left  | (0, 512)      | (2016, 19968)  | `..._x2016_y19968_patchXXXXX.png` |
| Bottom-Right | (512, 512)    | (2528, 19968)  | `..._x2528_y19968_patchXXXXX.png` |


In [1]:
import os
import h5py
import openslide
from PIL import Image
import numpy as np
import cv2
import csv

def get_white_pixel_ratio(pil_img, satThresh=30, valThresh=200):
    if pil_img.mode != 'RGB':
        pil_img = pil_img.convert('RGB')

    img_np = np.array(pil_img, dtype=np.uint8)
    hsv_img = cv2.cvtColor(img_np, cv2.COLOR_RGB2HSV)

    # Mask: low saturation AND high brightness (value)
    sat_mask = hsv_img[:, :, 1] < satThresh
    val_mask = hsv_img[:, :, 2] > valThresh
    white_mask = np.logical_and(sat_mask, val_mask)

    return np.sum(white_mask) / white_mask.size


def create_patches_from_h5(h5_path, svs_path, output_dir, skip_log_path):
    os.makedirs(output_dir, exist_ok=True)

    BASE_PATCH_SIZE = 1024
    SUB_PATCH_SIZE = 512
    base_name = os.path.splitext(os.path.basename(h5_path))[0]

    with h5py.File(h5_path, 'r') as h5_file:
        coords = h5_file['coords'][:]

    slide = openslide.OpenSlide(svs_path)
    patch_count = 0

    log_exists = os.path.exists(skip_log_path)
    
    # ✅ Open log file in write mode and keep it open during patch loop
    with open(skip_log_path, 'a', newline='') as log_file:
        writer = csv.writer(log_file)
        if not log_exists:
            writer.writerow(["PatchName", "X", "Y", "WhiteRatio"])  # Write header only once

        for i, (x, y) in enumerate(coords):
            base_patch = slide.read_region((int(x), int(y)), 0, (BASE_PATCH_SIZE, BASE_PATCH_SIZE)).convert('RGB')

            for row in range(2):
                for col in range(2):
                    offset_x = col * SUB_PATCH_SIZE
                    offset_y = row * SUB_PATCH_SIZE

                    abs_x = int(x) + offset_x
                    abs_y = int(y) + offset_y

                    sub_patch = base_patch.crop((
                        offset_x, offset_y,
                        offset_x + SUB_PATCH_SIZE, offset_y + SUB_PATCH_SIZE
                    ))

                    white_ratio = get_white_pixel_ratio(sub_patch)
                    patch_name = f"{base_name}_x{abs_x}_y{abs_y}_patch{patch_count:05d}.png"

                    if white_ratio > 0.8:
                        writer.writerow([patch_name, abs_x, abs_y, round(white_ratio, 4)])
                        continue

                    sub_patch.save(os.path.join(output_dir, patch_name))
                    patch_count += 1

    print(f"✅ Done! Saved {patch_count} coordinate-named 512x512 patches to {output_dir}")